In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import numpy as np
import io
import pandas as pd
import json
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding, Dense, Conv1D, MaxPooling1D, Flatten
from sklearn import preprocessing
import pickle
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kenne\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kenne\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Import Metadata CSV

In [2]:
metadata = pd.read_csv(os.getcwd() + '\movies_metadata.csv')

C:\Users\kenne\miniconda3\envs\hoMachineLearning\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
metadata.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [4]:
input_data = pd.DataFrame()
input_data['id'] = metadata['id'].copy()
input_data['title'] = metadata['title'].copy()
input_data['genres'] = metadata['genres'].copy()

In [5]:
input_data['genre'] = ''
input_data

,id,title,genres,genre
0,862,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",
1,8844,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",
2,15602,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",
3,31357,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",
4,11862,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",
...,...,...,...,...
45461,439050,Subdue,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",
45462,111109,Century of Birthing,"[{'id': 18, 'name': 'Drama'}]",
45463,67758,Betrayal,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",
45464,227506,Satan Triumphant,[],


In [6]:
### Just take first word
for i in range(input_data.shape[0]):
    
    json_str = input_data.iloc[i,2]
    json_str = json_str.strip('[]')
    json_str = json_str.replace('{','')
    json_str = json_str.replace('}','')
    json_list = json_str.split(',')
    for j in json_list[1:3:2]:
        word = j.split(':')[1].strip(' ').strip('\'')
    
    #words = word.strip(' ')
    input_data.iloc[i,3] = word

In [7]:
del input_data['genres']
input_data

,id,title,genre
0,862,Toy Story,Animation
1,8844,Jumanji,Adventure
2,15602,Grumpier Old Men,Romance
3,31357,Waiting to Exhale,Comedy
4,11862,Father of the Bride Part II,Comedy
...,...,...,...
45461,439050,Subdue,Drama
45462,111109,Century of Birthing,Drama
45463,67758,Betrayal,Action
45464,227506,Satan Triumphant,Action


### Import Keywords CSV

In [8]:
keywords = pd.read_csv(os.getcwd() + '\keywords.csv')

In [9]:
keywords.columns

Index(['id', 'keywords'], dtype='object')

In [10]:
keywords

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...
46414,439050,"[{'id': 10703, 'name': 'tragic love'}]"
46415,111109,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46416,67758,[]
46417,227506,[]


In [11]:
keywords['keywords_string'] = ''

In [12]:
keywords.head

<bound method NDFrame.head of            id                                           keywords  \
0         862  [{'id': 931, 'name': 'jealousy'}, {'id': 4290,...   
1        8844  [{'id': 10090, 'name': 'board game'}, {'id': 1...   
2       15602  [{'id': 1495, 'name': 'fishing'}, {'id': 12392...   
3       31357  [{'id': 818, 'name': 'based on novel'}, {'id':...   
4       11862  [{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...   
...       ...                                                ...   
46414  439050             [{'id': 10703, 'name': 'tragic love'}]   
46415  111109  [{'id': 2679, 'name': 'artist'}, {'id': 14531,...   
46416   67758                                                 []   
46417  227506                                                 []   
46418  461257                                                 []   

      keywords_string  
0                      
1                      
2                      
3                      
4                      
...      

In [13]:
for i in range(keywords.shape[0]):
    #for i in range(1):
    words = ''
    json_str = keywords.iloc[i,1]
    json_str = json_str.strip('[]')
    json_str = json_str.replace('{','')
    json_str = json_str.replace('}','')
    json_list = json_str.split(',')

    for j in json_list[1::2]:
        word = j.split(':')[1].strip(' ').strip('\'')
        words += word + ' '
    
    words = words.strip(' ')
    keywords.iloc[i,2] = words

In [14]:
del keywords['keywords']

In [15]:
keywords

,id,keywords_string
0,862,jealousy toy boy friendship friends rivalry bo...
1,8844,"board game disappearance ""based on children's ..."
2,15602,fishing best friend duringcreditsstinger old men
3,31357,based on novel interracial relationship single...
4,11862,baby midlife crisis confidence aging daughter ...
...,...,...
46414,439050,tragic love
46415,111109,artist play pinoy
46416,67758,
46417,227506,


In [16]:
keywords.dtypes

id                  int64
keywords_string    object
dtype: object

### Concat Data

In [17]:
keywords['id'] = keywords['id'].astype(str)

In [18]:
keywords.dtypes

id                 object
keywords_string    object
dtype: object

In [46]:
movie_data = pd.merge(input_data, keywords, on='id')

In [47]:
nan_value = float("NaN")
movie_data.replace("", nan_value, inplace=True)
movie_data.dropna(subset = ["title", "genre", "keywords_string"], inplace=True)

In [48]:
movie_data

,id,title,genre,keywords_string
0,862,Toy Story,Animation,jealousy toy boy friendship friends rivalry bo...
1,8844,Jumanji,Adventure,"board game disappearance ""based on children's ..."
2,15602,Grumpier Old Men,Romance,fishing best friend duringcreditsstinger old men
3,31357,Waiting to Exhale,Comedy,based on novel interracial relationship single...
4,11862,Father of the Bride Part II,Comedy,baby midlife crisis confidence aging daughter ...
...,...,...,...,...
46472,84419,House of Horrors,Horror,revenge murder serial killer new york city scu...
46473,390959,Shadow of the Blair Witch,Mystery,blair witch
46474,289923,The Burkittsville 7,Horror,witch mythology legend serial killer mockumentary
46477,439050,Subdue,Drama,tragic love


### Create Test Data

In [21]:
movie_data = movie_data.sample(frac=1).reset_index(drop=True)

In [22]:
#movie_data, test_movie_data = movie_data.iloc[:30656,:], movie_data.iloc[30656:,:]

In [24]:
#test_movie_data.to_csv('movie_data_test.csv')

### Create Input Strings

In [25]:
movie_data['input'] = movie_data['title'] + ' keywords ' + movie_data['keywords_string']

In [26]:
movie_data.drop(labels=['title', 'keywords_string', 'id'], axis=1, inplace=True)

 ### Preprocessing

In [27]:
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

In [28]:
def clean_text(text):
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [word for word in text if  not word in stop_words]
    text = " ".join(text)
    return text

In [29]:
movie_data['Processed_input'] = movie_data.input.apply(lambda x: clean_text(x))

In [30]:
X = movie_data['Processed_input']

In [31]:
Y = movie_data['genre']
le = preprocessing.LabelEncoder()
le.fit(movie_data['genre'])
Y = le.transform(movie_data['genre'])
max(Y)

19

In [32]:
with open('Label_encoder.pickle', 'wb') as handle:
    pickle.dump(le, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [33]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(movie_data['Processed_input'])
encoded_docs = tokenizer.texts_to_sequences(movie_data['Processed_input'])

In [34]:
# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [35]:
# pad sequences
max_length = max([len(s.split()) for s in movie_data['Processed_input']])
X = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(max_length)

193


### Split Data

In [36]:
X_train, X_val = X[:29656,:], X[29656:,:]

In [37]:
Y_train, Y_val = Y[:29656], Y[29656:]

### Build Neural Net

In [38]:
embeddings_dict = {}
with open("glove.6B.200d.txt", 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

In [39]:
vocab_size = len(tokenizer.word_index) + 1

In [40]:
embedding_matrix = np.zeros((vocab_size, 200))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [41]:
model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=200,
                           weights = [embedding_matrix],
                           input_length=max_length,
                           trainable=True))
#model.add(layers.Conv1D(filters=256, kernel_size=3, activation='relu'))
#model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(20, activation='sigmoid'))

In [42]:
# compile network
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam', metrics=['accuracy'])

In [43]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 193, 200)          4449200   
_________________________________________________________________
flatten (Flatten)            (None, 38600)             0         
_________________________________________________________________
dense (Dense)                (None, 20)                772020    
Total params: 5,221,220
Trainable params: 5,221,220
Non-trainable params: 0
_________________________________________________________________


In [44]:
# Train model with early stopping and save best model
past_accuracy = 0
session = 0
while True:
    model.fit(X_train, Y_train, batch_size=128, epochs=1, verbose=2)
    loss, accuracy = model.evaluate(X_val, Y_val)
    if past_accuracy < accuracy:
        past_accuracy = accuracy
        model_save = model
    else:
        session += 1
        if session == 3:
            break

224/224 - 17s - loss: 2.1986 - accuracy: 0.2948
63/63 [==============================] - 0s 5ms/step - loss: 1.9228 - accuracy: 0.3820
224/224 - 16s - loss: 1.6548 - accuracy: 0.4717
63/63 [==============================] - 0s 5ms/step - loss: 1.7514 - accuracy: 0.4595
224/224 - 16s - loss: 1.1896 - accuracy: 0.6341
63/63 [==============================] - 0s 5ms/step - loss: 1.7760 - accuracy: 0.4515
224/224 - 16s - loss: 0.7925 - accuracy: 0.7684
63/63 [==============================] - 0s 5ms/step - loss: 1.8862 - accuracy: 0.4455
224/224 - 16s - loss: 0.5197 - accuracy: 0.8568
63/63 [==============================] - 0s 5ms/step - loss: 2.0077 - accuracy: 0.4465


In [45]:
model_save.save(os.getcwd() + '\movie_genre_model')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: C:\Users\kenne\Documents\hoMachineLearning\TrueLayer\Training\movie_genre_model\assets
